# chapter 4. Classification

## Bayesian Optimization

### 베이지안 최적화 개요

**베이지안 최적화**는 목적 함수 식을 제대로 알 수 없는 블랙 박스 형태의 함수에서, 최대 또는 최소 함수 반환 값을 만드는 최적 입력값을 가능한 적은 시도를 통해 빠르고 효과적으로 찾아주는 방법이다. <br>
**베이지안 확률에 기반**을 두고 있는 최적화 기법으로, **새로운 데이터를 입력받았을 때 최적 함수를 예측하는 사후 모델을 개선해나가면서 최적 함수 모델을 만들어 낸다.** <br>
베이지안 최적화를 구성하는 두 가지 중요 요소는 **대체 모델(Surrogate Model)** 과 **획득 함수(Acquisition Function)** 이다. <br>
대체 모델은 획득 함수로부터 최적 함수를 예측할 수 있는 입력값을 추천 받은 뒤 이를 기반으로 최적 함수 모델을 개선해 나가며, 획득 함수는 개선된 대체 모델을 기반으로 최적 입력값을 계산한다. <br>

**베이지안 최적화 단계** <br>
1. 랜덤하게 하이퍼 파라미터를 샘플링, 성능 결과를 관측한다. <br>
2. 관측된 값을 기반으로 대체 모델은 최적 함수를 추정한다. <br>
3. 추정된 최적 함수를 기반으로 획득 함수는 다음으로 관측할 하이퍼 파라미터 값을 계산한다. <br>
4. 획득 함수로부터 전달된 하이퍼 파라미터를 수행하여 관측된 값을 기반으로 대체 모델은 갱신되어 다시 최적 함수를 예측 추정한다. <br>

대체 모델은 최적 함수를 추정할 때 일반적으로 가우시안 프로세스(Gaussian Process)를 적용한다. 

### HyperOpt

베이지안 최적화를 머신러닝 모델의 하이퍼 파라미터 튜닝에 적용할 수 있게 제공되는 여러 파이썬 패키지들이 있는데, 대표적으로 HyperOpt, Bayesian Optimization, Optuna 등이 있다. <br>

**HyperOpt를 활용하는 주요 로직 구성** <br>
1. 입력 변수명, 입력값의 검색 공간을 설정한다. <br>
2. 목적 함수의 설정 <br>
3. 목적 함수의 반환 최소값을 가지는 최적 입력값을 유추한다. <br>

**HyperOpt를 사용할 때 유의할 점은 다른 패키지와 다르게 목적 함수 반환 값이 최대값이 아닌 최소값을 가지는 최적 입력값을 유추한다는 것이다.** 

In [2]:
from hyperopt import hp

# -10 ~ 10까지 1 간격을 가지는 입력 변수 x와 -15 ~ 15까지 1 간격으로 입력 변수 y를 설정
search_space = {'x' : hp.quniform('x', -10, 10, 1), 'y' : hp.quniform('y', -15, 15, 1)}

**입력값의 검색 공간을 제공하는 대표적 함수**

|함수|내용|
|:------|:---|
|hp.quniform(label, low, high, q)|lable로 지정된 입력값 변수 검색 공간을 최소값 low에서 최대값 high까지 q의 간격을 가지고 설정한다. |
|hp.uniform(label, low, high)|최소값 low에서 최대값 high까지 정규 분포 형태의 검색 공간 설정 |
|hp.randint(label, low, high)|exp(uniform(low, high)) 값을 반환하며 반환 값의 log 변환된 값은 정규 분포 형태를 가지는 검색 공간 설정|
|hp.choice(label, options)|검색 값이 문자열 또는 문자열과 숫자값이 섞여 있을 경우 설정한다. |

목적 함수는 반드시 변수값과 검색 공간을 가지는 딕셔너리를 인자로 받고, 특정 값을 반환하는 구조로 만들어져야 한다. 

In [3]:
from hyperopt import STATUS_OK

# 목적 함수를 생성, 변수값과 변수 검색 공간을 가지는 딕셔너리를 인자로 받고 특정 값을 반환한다.
def objective_func(search_space):
    x = search_space['x']
    y = search_space['y']
    
    retval = x**2 - 20*y
    
    return retval

목적 함수의 반환값이 최소가 될 수 있는 최적의 입력값을 베이지안 최적화 기법에 기반하여 찾아야 한다. <br>
HyperOpt는 fmin(objective, space, algo, max_evals, trials)함수를 제공한다. <br>

**fmin() 주요 인자**

|함수|내용|
|:------|:---|
|fn|목적 함수|
|space|검색 공간 딕셔너리|
|algo|베이지안 최적화 적용 알고리즘, 기본적으로 tpe.suggest이며 이는 HyperOpt의 기본 최적화 알고리즘인 TPE(Tree of Parzen Estimator)를 의미한다. |
|max_evals|최적 입력값을 찾기 위한 입력값 시도 횟수이다. |
|trials|최적 입력값을 찾기 위해 시도한 입력값 및 해당 입력값의 목적 함수 반환값 결과를 저장하는데 사용한다. |
|rstate|fmin()을 수행할 때마다 동일한 결과값을 가질 수 있도록 설정하는 랜덤 시드 값이다. |

In [5]:
import numpy as np
from hyperopt import fmin, tpe, Trials

# 입력 결과값을 저장한 Trials 객체값 생성
trial_val = Trials()

# 목적 함수의 최소값을 반환하는 최적 입력 변수값을 5번의 입력값 시도(max_evals = 5)로 찾아냈다.
best_01 = fmin(fn = objective_func, space = search_space, algo = tpe.suggest, max_evals = 5, trials = trial_val, rstate = np.random.default_rng(seed = 0))

print('best : ', best_01)

100%|██████████████████████| 5/5 [00:00<00:00, 214.12trial/s, best loss: -224.0]
best :  {'x': -4.0, 'y': 12.0}


In [6]:
# 입력 결과값을 저장한 Trials 객체값 생성
trial_val = Trials()

# max_evals = 20로 변경해 다시 테스트
best_02 = fmin(fn = objective_func, space = search_space, algo = tpe.suggest, max_evals = 20, trials = trial_val, rstate = np.random.default_rng(seed = 0))

print('best : ', best_02)

100%|████████████████████| 20/20 [00:00<00:00, 533.60trial/s, best loss: -296.0]
best :  {'x': 2.0, 'y': 15.0}


fmin() 함수 수행 시 인자로 들어가는 Trials 객체는 함수의 반복 수행시마다 입력되는 변수값들과 함수 반환값을 속성으로 가지고 있다. <br>
**Trials 객체의 중요 속성**은 **results와 vals**가 있다. 

In [7]:
# fmin()에 인자로 들어가는 Trials 객체의 result 속성에 파이썬 리스트로 목적 함수 반환값들이 저장된다.
# 리스트 내부의 개별 원소는 {'loss' : 함수 반환값, 'status' : 반환 상태값}과 같은 딕셔너리이다.
print(trial_val.results)

[{'loss': -64.0, 'status': 'ok'}, {'loss': -184.0, 'status': 'ok'}, {'loss': 56.0, 'status': 'ok'}, {'loss': -224.0, 'status': 'ok'}, {'loss': 61.0, 'status': 'ok'}, {'loss': -296.0, 'status': 'ok'}, {'loss': -40.0, 'status': 'ok'}, {'loss': 281.0, 'status': 'ok'}, {'loss': 64.0, 'status': 'ok'}, {'loss': 100.0, 'status': 'ok'}, {'loss': 60.0, 'status': 'ok'}, {'loss': -39.0, 'status': 'ok'}, {'loss': 1.0, 'status': 'ok'}, {'loss': -164.0, 'status': 'ok'}, {'loss': 21.0, 'status': 'ok'}, {'loss': -56.0, 'status': 'ok'}, {'loss': 284.0, 'status': 'ok'}, {'loss': 176.0, 'status': 'ok'}, {'loss': -171.0, 'status': 'ok'}, {'loss': 0.0, 'status': 'ok'}]


In [8]:
# Trials 객체의 vals 속성에 {'입력 변수명' : 개별 수행 시마다 입력된 값 리스트} 형태로 저장된다.
print(trial_val.vals)

{'x': [-6.0, -4.0, 4.0, -4.0, 9.0, 2.0, 10.0, -9.0, -8.0, -0.0, -0.0, 1.0, 9.0, 6.0, 9.0, 2.0, -2.0, -4.0, 7.0, -0.0], 'y': [5.0, 10.0, -2.0, 12.0, 1.0, 15.0, 7.0, -10.0, 0.0, -5.0, -3.0, 2.0, 4.0, 10.0, 3.0, 3.0, -14.0, -8.0, 11.0, -0.0]}


vals는 딕셔너리 형태의 값을 가지며, 입력 변수 x와 y를 키 값으로 가지며, x와 y 키 값의 value는 20회의 반복 수행시마다 사용되는 입력값들을 리스트 형태로 가지고 있는 것을 알 수 있다. <br>

**Trials 객체의 results와 vals 속성**은 **HyperOpt의 fmin() 함수의 수행 시마다 최적화되는 경과를 볼 수 있는 함수 반환값**과 **입력 변수값들의 정보**를 제공한다. 

In [9]:
import pandas as pd

# results에서 loss 키 값에 해당하는 values를 추출해 리스트로 생성
losses = [loss_dict['loss'] for loss_dict in trial_val.results]

# DataFrame으로 생성
result_df = pd.DataFrame({'x' : trial_val.vals['x'], 'y' : trial_val.vals['y'], 'losses' : losses})
result_df

,x,y,losses
0,-6.0,5.0,-64.0
1,-4.0,10.0,-184.0
2,4.0,-2.0,56.0
3,-4.0,12.0,-224.0
4,9.0,1.0,61.0
5,2.0,15.0,-296.0
6,10.0,7.0,-40.0
7,-9.0,-10.0,281.0
8,-8.0,0.0,64.0
9,-0.0,-5.0,100.0


### HyperOpt를 이용한 XGBoost 하이퍼 파라미터 최적화 

**HyperOpt를 이용해 XGBoost의 하이퍼 파라미터를 최적화하는 방법** 

적용해야 할 하이퍼 파라미터와 검색 공간을 설정, 목적 함수에서 XGBoost를 학습 후 예측 성능 결과를 반환값으로 설정한다. <br>
fmin() 함수에서 목적 함수를 하이퍼 파라미터 검색 공간의 입력값들을 사용하여 최적의 예측 성능 결과를 반환하는 최적 입력값들을 결정하는 것이다. <br>
특정 하이퍼 파라미터들은 정수값만 입력을 받는데 HyperOpt는 입력값과 반환값이 모두 실수형이기 때문에 하이퍼 파라미터 입력 시 형변환을 해줘야하는 부분과 HyperOpt의 목적 함수는 최소값을 반환할 수 있도록 최적화 해야하기 때문에 성능 값이 클수록 좋은 성능 지표일 경우 -1을 곱해야 하는 것을 주의해야 한다.

In [4]:
# 이전 LightGBM Data
from lightgbm import LGBMClassifier

import pandas as pd
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

dataset = load_breast_cancer()

cancer_df = pd.DataFrame(data=dataset.data, columns = dataset.feature_names)

cancer_df['target'] = dataset.target
X_features = cancer_df.iloc[:, :-1]
Y_label = cancer_df.iloc[:, -1]

In [5]:
# 전체 데이터 중 80%는 학습용 데이터, 20%는 테스트용 데이터 추출
X_train, X_test, Y_train, Y_test = train_test_split(X_features, Y_label, test_size = 0.2, random_state = 156)
                                         
# X_train, Y_train을 다시 쪼개 90%은 학습과 10%은 검증용 데이터로 분리 
X_tr, X_val, Y_tr, Y_val = train_test_split(X_train, Y_train, test_size = 0.1, random_state = 156)

In [6]:
from hyperopt import hp

# max_depth는 5에서 20까지 1간격으로, min_child_weight는 1에서 2까지 1간격으로 
# colsample_bytree는 0.5에서 1에서, learning_rate는 0.01에서 0.2 사이 정규 분포된 값으로 검색한다. 
xgb_search_space = {'max_depth' : hp.quniform('max_depth', 5, 20, 1),
                    'min_child_weight' : hp.quniform('min_child_weight', 1, 2, 1), 
                    'learning_rate' : hp.uniform('learning_rate', 0.01, 0.2),
                    'colsample_bytree' : hp.uniform('colsample_bytree', 0.5, 1),
                    }

In [7]:
from sklearn.model_selection import cross_val_score
from xgboost import XGBClassifier
from hyperopt import STATUS_OK

# fmin()에서 입력된 search_space 값으로 입력된 모든 값은 실수형이다. 
# XGBClassifier의 정수형 하이퍼 파라미터는 정수형 변환을 해야 한다. 
# 정확도는 높을수록 더 좋은 수치이다. -1 * 정확도 -> 큰 정확도 값일수록 최소가 되도록 변환

def objective_func(search_space):
    # 수행 시간 절약을 위해 nestimators는 100으로 축소
    xgb_clf = XGBClassifier(n_estimators = 100, max_depth = int(search_space['max_depth']),
                           min_child_weight = int(search_space['min_child_weight']),
                           learning_rate = search_space['learning_rate'],
                           colsample_bytree = search_space['colsample_bytree'],
                           eval_metric = 'logloss')
    
    accuracy = cross_val_score(xgb_clf, X_train, Y_train, scoring = 'accuracy', cv = 3)
    
    # accuracy는 cv = 3 개수만큼 roc-auc 결과를 리스트로 가져간다. 이를 평균해 반환하되 -1을 곱한다. 
    return {'loss' : -1 * np.mean(accuracy), 'status' : STATUS_OK}

In [9]:
from hyperopt import fmin, tpe, Trials

trial_val = Trials()

best = fmin(fn = objective_func, 
           space = xgb_search_space,
           algo = tpe.suggest, 
           max_evals = 50, 
           trials = trial_val, rstate = np.random.default_rng(seed = 9))
print('best : ', best)

  0%|                                    | 0/50 [00:00<?, ?trial/s, best loss=?]

/Users/1001l1000/opt/anaconda3/envs/J/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)

/Users/1001l1000/opt/anaconda3/envs/J/lib/python3.9/site-packages/xgboost/data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):

/Users/1001l1000/opt/anaconda3/envs/J/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this 

  2%|▏        | 1/50 [00:00<00:16,  3.02trial/s, best loss: -0.9538892761705587]

/Users/1001l1000/opt/anaconda3/envs/J/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)

/Users/1001l1000/opt/anaconda3/envs/J/lib/python3.9/site-packages/xgboost/data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):

/Users/1001l1000/opt/anaconda3/envs/J/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this 

  4%|▎        | 2/50 [00:00<00:13,  3.68trial/s, best loss: -0.9560677355640758]

...

/Users/1001l1000/opt/anaconda3/envs/J/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)

/Users/1001l1000/opt/anaconda3/envs/J/lib/python3.9/site-packages/xgboost/data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):

/Users/1001l1000/opt/anaconda3/envs/J/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this 

 98%|███████▊| 49/50 [00:11<00:00,  3.94trial/s, best loss: -0.9692401533635412]

/Users/1001l1000/opt/anaconda3/envs/J/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)

/Users/1001l1000/opt/anaconda3/envs/J/lib/python3.9/site-packages/xgboost/data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):

/Users/1001l1000/opt/anaconda3/envs/J/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this 

100%|████████| 50/50 [00:12<00:00,  4.11trial/s, best loss: -0.9692401533635412]
best :  {'colsample_bytree': 0.548301545497125, 'learning_rate': 0.1840281762576621, 'max_depth': 18.0, 'min_child_weight': 2.0}


/Users/1001l1000/opt/anaconda3/envs/J/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)

/Users/1001l1000/opt/anaconda3/envs/J/lib/python3.9/site-packages/xgboost/data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):

/Users/1001l1000/opt/anaconda3/envs/J/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this 

In [10]:
print('colsample_bytree : {0}, learning_rate : {1}, max_depth : {2}, min_child_weight : {3}'.format(
round(best['colsample_bytree'], 5), round(best['learning_rate'], 5), 
int(best['max_depth']), int(best['min_child_weight'])))

colsample_bytree : 0.5483, learning_rate : 0.18403, max_depth : 18, min_child_weight : 2


In [12]:
xgb_wrapper = XGBClassifier(n_estimators = 400, 
                           learning_rate = round(best['learning_rate'], 5), 
                           max_depth = int(best['max_depth']),
                           min_child_weight = int(best['min_child_weight']),
                           colsample_bytree = round(best['colsample_bytree'], 5)
                           )

evals = [(X_tr, Y_tr), (X_val, Y_val)]
xgb_wrapper.fit(X_tr, Y_tr, early_stopping_rounds = 50, eval_metric = 'logloss', 
               eval_set = evals, verbose = True)

preds = xgb_wrapper.predict(X_test)
pred_proba = xgb_wrapper.predict_proba(X_test)[:, 1]

get_clf_eval(Y_test, preds, pred_proba)

[0]	validation_0-logloss:0.54472	validation_1-logloss:0.58675
[1]	validation_0-logloss:0.44155	validation_1-logloss:0.52577
[2]	validation_0-logloss:0.36542	validation_1-logloss:0.48906
[3]	validation_0-logloss:0.30756	validation_1-logloss:0.45704
[4]	validation_0-logloss:0.26142	validation_1-logloss:0.41671
[5]	validation_0-logloss:0.22616	validation_1-logloss:0.39605
[6]	validation_0-logloss:0.19465	validation_1-logloss:0.37095
[7]	validation_0-logloss:0.16951	validation_1-logloss:0.36066
[8]	validation_0-logloss:0.14718	validation_1-logloss:0.34686
[9]	validation_0-logloss:0.13006	validation_1-logloss:0.33716
[10]	validation_0-logloss:0.11635	validation_1-logloss:0.32332
[11]	validation_0-logloss:0.10455	validation_1-logloss:0.32073
[12]	validation_0-logloss:0.09388	validation_1-logloss:0.31916
[13]	validation_0-logloss:0.08434	validation_1-logloss:0.30988
[14]	validation_0-logloss:0.07702	validation_1-logloss:0.30469
[15]	validation_0-logloss:0.07144	validation_1-logloss:0.30293
[1

/Users/1001l1000/opt/anaconda3/envs/J/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Users/1001l1000/opt/anaconda3/envs/J/lib/python3.9/site-packages/xgboost/data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/Users/1001l1000/opt/anaconda3/envs/J/lib/python3.9/site-packages/xgboost/data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appro

[31]	validation_0-logloss:0.02961	validation_1-logloss:0.28505
[32]	validation_0-logloss:0.02881	validation_1-logloss:0.28319
[33]	validation_0-logloss:0.02801	validation_1-logloss:0.28407
[34]	validation_0-logloss:0.02710	validation_1-logloss:0.27684
[35]	validation_0-logloss:0.02654	validation_1-logloss:0.27598
[36]	validation_0-logloss:0.02591	validation_1-logloss:0.27329
[37]	validation_0-logloss:0.02514	validation_1-logloss:0.26722
[38]	validation_0-logloss:0.02484	validation_1-logloss:0.26668
[39]	validation_0-logloss:0.02424	validation_1-logloss:0.26733
[40]	validation_0-logloss:0.02404	validation_1-logloss:0.26721
[41]	validation_0-logloss:0.02374	validation_1-logloss:0.26519
[42]	validation_0-logloss:0.02351	validation_1-logloss:0.26809
[43]	validation_0-logloss:0.02295	validation_1-logloss:0.27016
[44]	validation_0-logloss:0.02272	validation_1-logloss:0.26844
[45]	validation_0-logloss:0.02252	validation_1-logloss:0.27126
[46]	validation_0-logloss:0.02233	validation_1-logloss:

[161]	validation_0-logloss:0.01342	validation_1-logloss:0.25930
[162]	validation_0-logloss:0.01339	validation_1-logloss:0.25879
[163]	validation_0-logloss:0.01336	validation_1-logloss:0.25875
[164]	validation_0-logloss:0.01333	validation_1-logloss:0.26031
[165]	validation_0-logloss:0.01331	validation_1-logloss:0.26007
[166]	validation_0-logloss:0.01329	validation_1-logloss:0.25962
[167]	validation_0-logloss:0.01326	validation_1-logloss:0.25964
[168]	validation_0-logloss:0.01324	validation_1-logloss:0.26111
[169]	validation_0-logloss:0.01321	validation_1-logloss:0.26049
[170]	validation_0-logloss:0.01319	validation_1-logloss:0.26053
[171]	validation_0-logloss:0.01316	validation_1-logloss:0.26029
[172]	validation_0-logloss:0.01314	validation_1-logloss:0.26079
[173]	validation_0-logloss:0.01312	validation_1-logloss:0.26035
[174]	validation_0-logloss:0.01310	validation_1-logloss:0.26013
[175]	validation_0-logloss:0.01308	validation_1-logloss:0.26039
[176]	validation_0-logloss:0.01306	valid

NameError: name 'get_clf_eval' is not defined